In [ ]:
import json
import pandas

In [ ]:
paper_json_path = "./paperData/1.json"
script_data_path = "./slideMeta/slideData/0/scriptData.txt"
ocr_data_path = "./slideMeta/slideData/0"

In [ ]:
with open(paper_json_path, "r") as json_file :
    paper_json_obj = json.load(json_file)
    
print(paper_json_obj)

In [ ]:
import re
from string import digits
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

# kjnltk.download()

def preprocess(t) :
    t=re.sub("\[.*?\]","",t)
    
    sentences = nltk.tokenize.sent_tokenize(t)
    
    # sentences = [ ' '.join(re.sub(r'[^\w]', ' ', s).split()) for s in sentences ]
    
    return sentences
    
def isAllowedSection(t) :
    if t == "REFERENCES" :
        return False
    else :
        return True

every_sentences = []
every_labels = []

def fixSectionTitles(section_titles):
    ret_titles = []
    title_num = 0
    last_section = None
    def is_main_section(title):
        if title[0] in digits or title.isupper() is True:
            return True
        return False
    for title in section_titles:
        if is_main_section(title) or last_section is None:
            ret_titles.append(title)
            last_section = title
        else:
            ret_titles.append(last_section)
    return ret_titles

for i in range(len(paper_json_obj['sections'])) :
    if 'title' in paper_json_obj['sections'][i] and 'text' in paper_json_obj['sections'][i]['title'] :
        sectionTitle = paper_json_obj['sections'][i]['title']['text']
    
        if not isAllowedSection(sectionTitle) :
            continue
            
        for p in range(len(paper_json_obj['sections'][i]['paragraphs'])) :
            bodyText = paper_json_obj['sections'][i]['paragraphs'][p]['text']
            
            bodyText = preprocess(bodyText.lower())
            
            every_labels.append(sectionTitle)
            every_sentences.append(sectionTitle.lower() + '.')

            every_labels = [*every_labels, *[sectionTitle for i in range(len(bodyText)) ] ]
            every_sentences = [*every_sentences, *bodyText]
            
            
every_labels = fixSectionTitles(every_labels)

__every_labels = []
__every_sentences = []

for i in range(len(every_labels)) :
    if every_labels[i].split(' ')[0].isdigit() :
        __every_labels.append(every_labels[i])
        __every_sentences.append(every_sentences[i])
        
every_labels = __every_labels
every_sentences = __every_sentences

for i, sentence in enumerate(every_sentences):
    if every_labels[i].startswith('7'):
        print(sentence)

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1, 3))
X = vectorizer.fit_transform(every_sentences)

print(vectorizer.get_feature_names_out())

print(X.shape)

In [ ]:
import numpy as np

label_dict = sorted(list(set(every_labels)))
label_categories = [ label_dict.index(s) for s in every_labels ]

Y = np.array(*[label_categories])

Y.shape

In [ ]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
# define dataset
model = RandomForestClassifier()

model.fit(X, Y)

# from sklearn.datasets import load_iris
# from sklearn.linear_model import LogisticRegression

# model = LogisticRegression()

# model.fit(X, Y)

In [ ]:
def top_k_predictions(t, k):
    pre = ' '.join(preprocess(t))
    X2 = vectorizer.transform([pre])
    all_probs = model.predict_proba(X2)[0]
    args = np.argsort(all_probs)[-k:][::-1]
    ret = {}
    
    for arg in args:
        ret[label_dict[arg]] = all_probs[arg]
        
    return ret

def makePrediction(t) :
    pre = ' '.join(preprocess(t))
    
    X2 = vectorizer.transform([pre])
    
    return [ label_dict[s] for s in model.predict(X2) ]

def makePredictionProbability(t) :
    pre = ' '.join(preprocess(t))
    
    X2 = vectorizer.transform([pre])
    
    return model.predict_proba(X2)[0]
    
p = "how people use career mentoring relevant to their careers, and we will be focusing on the online communities as a source of mentoring. And the main research question we're asking is how people seek and receive advice."

result = makePredictionProbability(p)

print(result)
    

In [ ]:
import os

scriptDataFile = open(script_data_path, "r")
cnt = 0

scriptList = []

while True:
    line = scriptDataFile.readline()
    
    if not line: 
        break
        
    raw = line.strip()
    scriptList.append(raw)
    # print(raw)
    # print('')

ocrResult = []
for i in range(len(scriptList)) :
    ocrFile = open(os.path.join(ocr_data_path, "ocr", str(i) + ".jpg.txt"), "r")
    ocrResult.append('')
    firstLine = True
    while True :
        line = ocrFile.readline()
        if not line :
            break
        if firstLine:
            firstLine = False
            continue
        res = line.split('\t')[-1].strip()
        if len(res) > 0:
            ocrResult[-1] = ocrResult[-1] + ' ' + res

In [ ]:
__cnt = 0

thresh = 3
val_thresh = 0.1

probTable = []
topSections = []

# print(scriptList)

for i in range(len(scriptList)):
    scriptList[i] = scriptList[i] + " " + ocrResult[i]

for i in range(len(scriptList)) :
    tokens = nltk.tokenize.word_tokenize(scriptList[i])
    
    ### BLURRING

    # sent = ''
    
    # if len(tokens) < 25 :
    #     sent = '.'.join(scriptList[max(-2+i, 0):i])
    # else :
    #     sent = scriptList[i]
        
    # preds = top_k_predictions(sent, len(label_dict))

    # topSections.append(preds)
    # probTable.append(makePredictionProbability(sent))


    # ### THRESHOLDING, BLURRING with PREVIOUS
    # preds = top_k_predictions(scriptList[i], len(label_dict))
    # if len(tokens) < 25 and len(topSections) > 0:
    #     topSections.append(topSections[-1])
    # else:
    #     vals = list(preds.values())
    #     prob = vals[np.argsort(vals)[-thresh]]
    #     prob = max(prob, val_thresh)
    #     for k in preds.keys():
    #         if preds[k] < prob:
    #             preds[k] = 0
    #     topSections.append(preds)
    # probTable.append(makePredictionProbability(scriptList[i]))

    # ### BLURRING with PREVIOUS
    # preds = top_k_predictions(scriptList[i], len(label_dict))
    # if len(tokens) < 25 and len(topSections) > 0:
    #     topSections.append(topSections[-1])
    # else:
    #     # vals = list(preds.values())
    #     # prob = vals[np.argsort(vals)[-thresh]]
    #     # prob = max(prob, val_thresh)
    #     # for k in preds.keys():
    #     #     if preds[k] < prob:
    #     #         preds[k] = 0
    #     topSections.append(preds)
    # probTable.append(makePredictionProbability(scriptList[i]))


    ### NO BLURRING
    preds = top_k_predictions(scriptList[i], len(label_dict))
    # vals = list(preds.values())
    # prob = vals[np.argsort(vals)[-thresh]]
    # prob = max(prob, val_thresh)
    # for k in preds.keys():
    #     if preds[k] < prob:
    #         preds[k] = 0
    topSections.append(preds)
    probTable.append(makePredictionProbability(scriptList[i]))


In [ ]:
__topSections = []
for topSection in topSections:
    __topSection = []
    for i, (k, v) in enumerate(topSection.items()):
        __topSection.append([k, i, v])
    __topSections.append(__topSection)

topSections = __topSections


In [ ]:
SKIPPED_SECTIONS = [
    "CCS CONCEPTS",
    "KEYWORDS",
    "ACM Reference Format:",
    "REFERENCES",
    "ACKNOWLEDGMENTS"
]

def is_section_skipped(section):
    for skipped_section in SKIPPED_SECTIONS:
        if skipped_section in section:
            return True
    return False

def getOutlineHyungyu(sectionData, topSections, script_sentence_range):
    uniqueSections = []
    for section in sectionData:
        if is_section_skipped(section) or section in uniqueSections:
            continue
        uniqueSections.append(section)

    INF = (len(script_sentence_range) + 1) * 100
    n = len(uniqueSections)

    Table = [ [ (-INF, n, -1) for j in range(len(script_sentence_range))] for i in range(len(script_sentence_range)) ]

    def getSegment(start, end):
        if end - start < 2:
            return (-INF, n)

        scores = [0 for i in range(n)]
        for i in range(start, end):
            innerScores = [-INF for i in range(n)]
            for topSection in topSections[i]:
                pos = uniqueSections.index(topSection[0])
                innerScores[pos] = max(innerScores[pos], topSection[2])
            for j in range(n):
                if innerScores[j] > 0:
                    scores[j] += innerScores[j]
        result_section = -1
        for i in range(n):
            if result_section == -1 or scores[result_section] < scores[i]:
                result_section = i

        return (scores[result_section], result_section)

    Table[0][0] = (0, n, 0)

    for i in range(1, len(script_sentence_range)):
        segResult = getSegment(i, i + 1)

        Table[i][i] = (max(Table[i-1][i-1][0] + segResult[0], Table[i][i][0]), segResult[1], i)

        for j in range(i+1, len(script_sentence_range)) :
            for k in range(i-1, j) :
                cost = getSegment(k+1, j+1)
                if Table[i][j][0] < Table[i-1][k][0] + cost[0]:
                    Table[i][j] = (Table[i-1][k][0] + cost[0], cost[1], k + 1)

    weight = []
    for i in range(len(script_sentence_range)) :
        weight.append(Table[i][len(script_sentence_range) - 1][0])

    myMaxValue = -INF
    optSegs = -1

    for i in range(len(Table)) :
        if myMaxValue < Table[i][len(script_sentence_range) - 1][0]:
            myMaxValue = Table[i][len(script_sentence_range) - 1][0]
            optSegs = i

    finalResult = [ 0 for i in range(len(script_sentence_range)) ]

    curSlide = len(script_sentence_range) - 1
    print(myMaxValue, optSegs)

    while optSegs > 0:
        start = Table[optSegs][curSlide][2]
        curSection = Table[optSegs][curSlide][1]

        for i in range(start, curSlide + 1) :
            finalResult[i] = curSection
        
        curSlide = start - 1
        optSegs = optSegs - 1

        if curSlide < 0 :
            print("WHAT???? ERROR ERROR ERROR ERROR")
            break

    outline = []

    outline.append({
        'section': "NO_SECTION",
        'startSlideIndex': 0,
        'endSlideIndex': 0
    })

    for i in range(1, len(finalResult)) :
        if outline[-1]['section'] != uniqueSections[finalResult[i]]:
            outline.append({
                'section': uniqueSections[finalResult[i]],
                'startSlideIndex': i,
                'endSlideIndex': i
            })
        else :
            outline[-1]['endSlideIndex'] = i
    return outline, weight

def getOutlineMaskDP(sectionData, topSections, script_sentence_range, target_mask = None):
    uniqueSections = []
    for section in sectionData:
        if is_section_skipped(section) or section in uniqueSections:
            continue
        uniqueSections.append(section)

    for i, section in enumerate(uniqueSections):
        print(i, ":", section)

    n = len(uniqueSections)
    m = len(script_sentence_range)
    INF = (m + 1) * 100

    dp = [[(-INF, n, -1) for j in range(m)] for i in range(1 << n)]
    dp[0][0] = (0, n)

    for i in range(m):
        scores = [0 for k in range(n)]
        for j in range(i + 1, m):
            for topSection in topSections[j]:
                pos = uniqueSections.index(topSection[0])
                scores[pos] += topSection[2]

            for mask in range(0, (1 << n)):
                if dp[mask][i][0] < 0:
                    continue
                for k in range (n):
                    if mask & (1 << k) > 0:
                        continue
                    nmask = mask | (1 << k)
                    if (dp[nmask][j][0] < dp[mask][i][0] + scores[k]):
                        dp[nmask][j] = (dp[mask][i][0] + scores[k], k, i)

    recoverMask = target_mask
    recoverSlideId = m - 1


    if recoverMask is None or dp[recoverMask][recoverSlideId][0] < 0:
        recoverMask = 0
        for mask in range(1 << n):
            #print(mask, dp[mask][recoverSlideId])
            if dp[mask][recoverSlideId][0] > dp[recoverMask][recoverSlideId][0] \
                or (bin(mask).count('1') < bin(recoverMask).count('1') and dp[mask][recoverSlideId][0] == dp[recoverMask][recoverSlideId][0]
            ):
                recoverMask = mask

    if target_mask is not None:
        print (bin(target_mask), bin(recoverMask), dp[target_mask][recoverSlideId][0])
    
    outline = []
    while recoverSlideId > 0:
        print(recoverMask, recoverSlideId, dp[recoverMask][recoverSlideId])
        sectionId = dp[recoverMask][recoverSlideId][1]
        nextRecoverMask = recoverMask ^ (1 << sectionId)
        nextRecoverSlideId = dp[recoverMask][recoverSlideId][2]
        outline.append({
            "section": uniqueSections[sectionId],
            "startSlideIndex": nextRecoverSlideId + 1,
            "endSlideIndex": recoverSlideId,
        })
        recoverMask = nextRecoverMask
        recoverSlideId = nextRecoverSlideId
    
    return outline[::-1]

In [ ]:
#target_mask = int('0b00011111', 2)
target_mask = None

outline = (getOutlineMaskDP(label_dict, topSections, scriptList, target_mask))

for single in outline:
    print(single)



In [ ]:
outline, weights = (getOutlineHyungyu(label_dict, topSections, scriptList))

for bullet in outline:
    print(bullet)



In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

sns.heatmap(probTable[47:51:1])

print(47, 48, 49, 50, 51)
for i in range(47, 51):
    print(probTable[i])

plt.title('Annoteat cell with numeric value', fontsize=20)
plt.show()


for e in enumerate(label_dict) :
    print(e)

In [ ]:
def fixSectionTitles(section_titles):
    ret_titles = []
    title_num = 0
    last_section = None
    def is_main_section(title):
        if title[0] in digits or title.isupper() is True:
            return True
        return False
    for title in section_titles:
        if is_main_section(title) or last_section is None:
            ret_titles.append(title)
            last_section = title
        else:
            ret_titles.append(last_section)
    print(ret_titles)
    return ret_titles
